In [1]:
import pandas as pd
import os
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from fuzzywuzzy import fuzz


# Function to check and download necessary NLTK resources
def download_nltk_resources():
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')

    try:
        nltk.data.find('taggers/averaged_perceptron_tagger')
    except LookupError:
        nltk.download('averaged_perceptron_tagger')

    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords')

# Call the function to ensure resources are downloaded
download_nltk_resources()


In [2]:
PATH = os.getcwd()
PATH = PATH + '/data'

def load_data(file):
    file_path = os.path.join(PATH,file)
    file = pd.read_csv(file_path)
    return file
product_cat=load_data('categories.csv')
product_cat = product_cat.drop('CATEGORY_ID',axis=1)

brand_cat  = load_data('brand_category.csv')
brand_cat = brand_cat.drop('RECEIPTS',axis=1)

retailers = load_data('offer_retailer.csv')

In [3]:
# Clean the foreign keys 
def clean_str(df,cols):
    if isinstance(cols,list):
        for col in cols:
            df[col] = df[col].map(lambda x: x.strip().upper() if isinstance(x, str) else x)
    elif isinstance(cols,str):
        df[col] = df[col].map(lambda x: x.strip().upper() if isinstance(x, str) else x)
    else:
        raise KeyError

In [4]:
clean_str(product_cat, ['PRODUCT_CATEGORY', 'IS_CHILD_CATEGORY_TO'])
clean_str(brand_cat, ['BRAND','BRAND_BELONGS_TO_CATEGORY'])
clean_str(retailers,['RETAILER','BRAND'])

In [5]:
product_cat_refined = product_cat.merge(product_cat[['PRODUCT_CATEGORY','IS_CHILD_CATEGORY_TO']], left_on='IS_CHILD_CATEGORY_TO',right_on='PRODUCT_CATEGORY' ,how='left',suffixes=('','_compli'))
product_cat_refined = product_cat_refined.drop("PRODUCT_CATEGORY_compli",axis=1)

In [6]:
product_cat_refined.isna().sum()

PRODUCT_CATEGORY                0
IS_CHILD_CATEGORY_TO            0
IS_CHILD_CATEGORY_TO_compli    83
dtype: int64

In [10]:
product_cat_refined['PRODUCT_CATEGORY'].value_counts().sort_values()

PRODUCT_CATEGORY
RED PASTA SAUCE            1
FEMININE HYGEINE           1
LEAFY SALADS               1
CREAM                      1
COFFEE                     1
                          ..
FROZEN BEEF                1
FROZEN SEAFOOD             1
BATH & BODY                1
FROZEN PLANT-BASED MEAT    1
GUM                        1
Name: count, Length: 118, dtype: int64

## add cat to Offer

In [12]:
cat_check = brand_cat.groupby("BRAND")["BRAND_BELONGS_TO_CATEGORY"].count()
unique_cat  = cat_check[cat_check<2]
dup_cat = cat_check[cat_check>=2]
unique_brand_cat = brand_cat[brand_cat['BRAND'].isin(unique_cat.index)]
unique_brand_cat

dup_brand_cat = brand_cat[brand_cat['BRAND'].isin(dup_cat.index)]
dup_brand_cat

combined_unique_brand = retailers.merge(unique_brand_cat, how='inner', on='BRAND')
combined_dup_brand  = retailers.merge(dup_brand_cat, how='inner', on='BRAND')

In [14]:
# def extract_offer_info(x):
#     offer_info= set(str(x['OFFER_clean']).lower().split(" "))
#     cat_info = set(str(x['BRAND_BELONGS_TO_CATEGORY']).lower().split(" "))

#     if str(x['BRAND_BELONGS_TO_CATEGORY']).lower() in str(x['OFFER']).lower():
#         return x['BRAND_BELONGS_TO_CATEGORY']
#     elif offer_info & (cat_info):
#         return x['BRAND_BELONGS_TO_CATEGORY']
#     else:
#         return np.nan

# combined_unique_brand['refined_cat'] = combined_unique_brand.apply(extract_offer_info,axis=1)

### Clean OFFER

In [15]:

# set(stopwords.words('english'))
def text_prep(txt,brand,retailer,stop_words):
    txt = re.sub(r'\$?\d+', '', txt)
    txt = txt.replace("'","").replace("$","").replace('-'," ")

    if pd.notna(brand) or pd.notna(retailer):
        brand_retailer = str(brand).replace("'","").title().split()+str(retailer).replace("'","").title().split()
        for b in brand_retailer :
            txt = txt.title().replace(b,"")
    words = word_tokenize(txt)
    # words =[word.strip(u"\u2122").strip(u'\u0256') for word in words]
    filtered_text = [word for word in words if (not word.lower() in stop_words) and (word.isalpha()) 
                     ]
    # tagged = pos_tag(filtered_text)
    # nouns = [word for word, pos in tagged if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    return ' '.join(filtered_text)

In [16]:
def dup_cat_clean(row):
    cat = row['BRAND_BELONGS_TO_CATEGORY'].lower().replace('&'," ").split()
    cat = ' '.join(cat)
    offer = row['OFFER_clean'].lower()
    fuzz_score1 = fuzz.partial_ratio(cat, offer)
    # fuzz_score1 = fuzz.token_sort_ratio(cat, offer)
    
    return fuzz_score1

In [17]:
sw = set(stopwords.words('english'))
sw.update({"buy", "spend", "select", 'varieties', 'sizes', 'ounce', 'count', 'liter'})
sw.remove('any')
brand_set = set(brand_cat['BRAND'].str.capitalize())
combined_dup_brand['OFFER_clean'] = combined_dup_brand.apply(lambda row: text_prep(row['OFFER'],row['BRAND'],row['RETAILER'],sw),axis=1)


In [18]:
##### add general offer to unique dataset

In [19]:
general_offer = combined_dup_brand[combined_dup_brand['OFFER_clean'].str.lower().str.contains("reward|club|member")]
unique_brand = pd.concat([combined_unique_brand,general_offer],axis=0)

In [20]:
##### filter duplicate offers

In [21]:
mislabeled_offer = combined_dup_brand[~combined_dup_brand['OFFER_clean'].str.lower().str.contains("reward|club|member")].copy()

In [22]:
mislabeled_offer['CAT_score'] = mislabeled_offer.apply(dup_cat_clean,axis=1)

In [24]:
final_dup = mislabeled_offer.groupby(['OFFER', 'BRAND']).apply(lambda x: x.loc[x['CAT_score'].idxmax()]).drop('CAT_score',axis=1)

In [25]:
final_dup = final_dup.reset_index(drop=True)

In [26]:
training  = pd.concat([unique_brand,final_dup],axis=0)

In [28]:
training = training.merge(product_cat_refined, how='left' , left_on="BRAND_BELONGS_TO_CATEGORY", right_on='PRODUCT_CATEGORY')

In [30]:
def join_str(row):
    s = f"""{str(row['OFFER_clean']) if pd.notna(row['OFFER_clean']) else " "} {str(row['BRAND']).replace('&'," ").replace(",", " ")} {str(row['RETAILER']) if pd.notna(row['RETAILER']) else " "} {row['PRODUCT_CATEGORY']}, {row['IS_CHILD_CATEGORY_TO_compli'] if pd.notna(row['IS_CHILD_CATEGORY_TO_compli']) else row['PRODUCT_CATEGORY']}"""
    return s

In [31]:
training['training_str'] = training.apply(join_str,axis=1).to_list()

In [83]:
training.to_csv(f"{PATH}/processed_data.csv", index=False)

In [9]:
sw = set(stopwords.words('english'))
sw.update({"buy", "spend", "select", 'varieties', 'sizes', 'ounce', 'count', 'liter'})
sw.remove('any')

def input_prep_fz(txt,stop_words):
    txt = re.sub(r'\$?\d+', '', txt)
    txt = txt.replace("'","").replace("$","").replace('-'," ")

    words = word_tokenize(txt)
    # words =[word.strip(u"\u2122").strip(u'\u0256') for word in words]
    filtered_text = [word for word in words if (not word.lower() in stop_words) and (word.isalpha())]
    tagged = pos_tag(filtered_text)
    nouns = [word for word, pos in tagged if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    return " ".join(nouns)

In [13]:
input_prep_fz("I want some milk, this is a nauns. ok, I want",sw)

'milk nauns'